### **Reading Data From Sources**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
#if Init_load_flag == 1:

In [0]:
Init_load_flag = int(dbutils.widgets.get("Init_load_flag"))
#Init_load_flag = int(dbutils.widgets.get("Init_load_flag"))

# **Reading data from the source**

In [0]:
# 1- Reading the data from source

df = spark.sql("select * from databricks_catalog.silver.customer_silver")

In [0]:
# 2-  Remove diuplicates based on the promary key: Customer ID
df = df.dropDuplicates(subset=["customer_id"])
df.limit(10).display()



**Dividing New vs Old Records**

In [0]:
if Init_load_flag == 0:

    df_old = spark.sql('''select DimCustomersKey, customer_id, create_date, update_date
                          from databricks_catalog.gold.DimCustomers''')

else:

    df_old = spark.sql('''select 0 DimCustomersKey, 0 customer_id, 0 create_date, 0 update_date
                          FROM databricks_catalog.silver.customer_silver where 1=0''')
                  

In [0]:
df_old.display()

**Rename Columns of df_old**

In [0]:
df_old = df_old.withColumnRenamed("DimCustomersKey", "old_DimCustomersKey")\
    .withColumnRenamed("customer_id", "old_customer_id")\
    .withColumnRenamed("create_date", "old_create_date")\
    .withColumnRenamed("update_date", "old_update_date")

#**Applying Join with the old records**

In [0]:

df_join = df.join(df_old, df['customer_id'] == df_old['old_customer_id'], 'left')
df_join.display()

**Seperating old vs new records**

In [0]:
df_new = df_join.filter(df_join["old_DimCustomersKey"].isNull())

In [0]:
df_old = df_join.filter(df_join["old_DimCustomersKey"].isNotNull())

**Preparing df_old**

In [0]:
 #droping columns that are not needed

df_old = df_old.drop("old_customer_id", "old_update_date")

#Renaming old_DimCustomersKey to DimCustomersKey
df_old = df_old.withColumnRenamed("old_DimCustomersKey","DimCustomersKey")

#Renaming old_create_date to create_date
df_old = df_old.withColumnRenamed("old_create_date","create_date")

#Recreateong update date with to_timestamp
df_old = df_old.withColumn("create_date",to_timestamp(col("create_date")))

#Recreateong update date with current_timestamp

df_old = df_old.withColumn("update_date", current_timestamp())


In [0]:
df_old.display()

**Preparing df New**

In [0]:
#droping columns that are not needed

df_new = df_new.drop("old_DimCustomersKey","old_customer_id", "old_update_date", "old_create_date")

#Recreateong "update_date" and "create_date" with current_timestamp

df_new = df_new.withColumn("update_date", current_timestamp())
df_new = df_new.withColumn("create_date", current_timestamp())

#Recreateong update date with to_timestamp
#df_new = df_new.withColumn("create_date",to_timestamp(col("create_date")))




In [0]:
df_new.display()

**Surrogate Key from 1**


In [0]:
# AI: 1- add this
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number


In [0]:
# 2- remove the code below
#df_new = df_new.withColumn("DimCustomersKey",monotonically_increasing_id()+lit(1))

In [0]:
#3- Replcae the code above with this below
# Define a window that orders rows by a stable column like customer_id
window_spec = Window.orderBy("customer_id")

# Create a local row number starting from 1
df_new = df_new.withColumn("local_id", row_number().over(window_spec))

In [0]:
df_new.display()

### **Add max Surrogate Key**

In [0]:

if Init_load_flag == 1:
 max_surrogate_key = 0

else:
    df_maxsur = spark.sql("select max(DimCustomersKey) as max_surrogate_key from databricks_catalog.gold.DimCustomers")
    #Converting df_maxsur to max_surrogate_key 
    max_surrogate_key = df_maxsur.collect()[0]["max_surrogate_key"]


In [0]:
#4 replace your code with mine second code below
#df_new = df_new.withColumn("DimCustomersKey",lit(max_surrogate_key)+col("DimCustomersKey"))
df_new = df_new.withColumn("DimCustomersKey", lit(max_surrogate_key) + col("local_id"))
df_new = df_new.drop("local_id")

In [0]:
df_new.display()


**#union of df_old and df_new**

In [0]:
df_final = df_new.unionByName(df_old)

In [0]:
df_final.display()


### **SCD Type 1**###

In [0]:
from delta.tables import DeltaTable

In [0]:
# instead of first line of code down velow: i can use the if Init... too: if Init_load_flag == 1:
if spark.catalog.tableExists("databricks_catalog.gold.DimCustomers"):
    dlt_obj = DeltaTable.forPath(spark, "abfss://gold@ssdatalakeeteproject.dfs.core.windows.net/DimCustomers")

    dlt_obj.alias("trg").merge(df_final.alias("src"), "trg.DimCustomersKey = src.DimCustomersKey") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

else:
    df_final.write.mode("overwrite") \
        .format("delta") \
        .option("path", "abfss://gold@ssdatalakeeteproject.dfs.core.windows.net/DimCustomers") \
        .saveAsTable("databricks_catalog.gold.DimCustomers")


In [0]:
%sql
select * from databricks_catalog.gold.DimCustomers

### Done To Here###
